# Отток клиентов


<div class="alert alert-info">
Привет! Меня зовут Никита Мишин и я буду твоим ревьюером по этому проекты.
Для простоты предлагаю общение на 'ты'. Буду предполагать, что ты не против:) 
Если более предпочтительно обращение на 'Вы', пиши, не стесняйся.
Также если будут возникать вопросы, аналогично, пиши:)

Предлагаю работать в известном тебе итеративном формате.
Итерация состоит в моей проверке твоего решения. 
После решения могут остаться какие-то недочеты, которые я попрошу тебя устранить, ты их исправляешь и я проверяю твои решения.
    Оставленные мною комментарии могут быть разного вида:
   
    - зеленый: элегантные решения, которые тебе стоит запомнит и в дальнейшем взять на вооружение:) 
    
    - желтый: сигнал о том, что есть некритичная вещь(не всегда ошибка), что нужно точно поправить в следующей работе, даже желательно в этой (полезно, в первую очередь, для тебя:) ).Также это рекомендации на будущее    

    - красный: недочет, который нужно исправить в этой работе, для того, чтобы она была принята
    
    - синий: полезная информация, доп ресурсы, "вопросы на подумать"

Также попрошу не удалять мои комментарии:) <a class="tocSkip">
</div>

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split  #Деление на выборки (тестовый набор данных)
from sklearn.metrics import accuracy_score #Метрика оценивания точности
from sklearn.preprocessing import OneHotEncoder #Кодирование категориальных функций

from sklearn.tree import DecisionTreeClassifier #Дерево решений
from sklearn.linear_model import LogisticRegression #Логистическая регрессия
from sklearn.ensemble import RandomForestClassifier #Модель случайного леса

from sklearn.utils import shuffle #перетасовать
from sklearn.metrics import confusion_matrix #Матрица ошибок
from sklearn.metrics import recall_score #Полнота
from sklearn.metrics import precision_score #Точность

from sklearn.metrics import f1_score #F1-мера
from sklearn.metrics import roc_auc_score #AUC-ROC

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Молодец, что сгруппировала все импорты в одну ячейку, так удобнее и легче вносить изменения. Пользователь тетрадки  сразу поймет, что отсутствуют какие-то библиотеки
</div>

In [2]:
churn_data = pd.read_csv('/datasets/Churn.csv')

display(churn_data.head(10))

print('=============================')
churn_data.info()

print('=============================')
print('Кол-во пропусков')
report = churn_data.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / churn_data.shape[0]).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
Кол-во пропусков


,missing_values,% of total
Tenure,909,0.09
RowNumber,0,0.00
CustomerId,0,0.00
Surname,0,0.00
CreditScore,0,0.00
Geography,0,0.00
Gender,0,0.00
Age,0,0.00
Balance,0,0.00
NumOfProducts,0,0.00


<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Можно не оборачивать последнюю команду в print, она же последняя в ячейке:)</div>

### Вывод

Обзор информация помог определить план дальнейшей работы:

1. Перевести столбцы в нижний регистр;
2. Провести кодирование признака gender; 
3. Заменим пропуски в столбце tenure на значение 0, т.е как пропуски составляют меньше 10%
4. Кодирование значение в категориальном столбце geography в числовое значение
4. Определить признаки (исключим из признаков столбцы surname, rownumber, customerid) и целевой признак;
5. Разделить набор на обучающий, валидационный, тестовый набор данных;
6. Дальнейшие исследования и выводы

<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Интересно было бы посмотреть на распределение признаков, матрицу корреляций и основные статистики
</div>

<div class="alert alert-danger">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Хм, пара моментов:
    
    1) Почему ты удалила признак geography? (разве он будет бесполез для наших моделей?)
    
    2) RowNumber и CustomerId идентификаторы, они как-то помогут модели найти скрытые зависимости?
    
</div>

In [3]:
#1. Перевести столбцы в нижний регистр
churn_data.columns = churn_data.columns.str.lower()

#2. Провести кодирование признака gender
churn_data['gender']= pd.get_dummies(churn_data['gender'])

# 3. Заменим пропуски в столбце tenure на значение 0
churn_data['tenure'] = churn_data['tenure'].fillna(0.0)

#Кодирование значение в категориальном столбце geography в числовое значение
labels = OneHotEncoder(handle_unknown='ignore')
churn_data_x = pd.DataFrame(labels.fit_transform(churn_data[['geography']]).toarray())
churn_data=churn_data.join(churn_data_x)

<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Элегантное решенеие с str.lower, молодец:)

... Да, замена на 0 выглядит вполне логичной и обоснованной
</div>

<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Вопрос на подумать: подходит ли LE?


Ответ:  Спасибо. очень хорошая штука. Незнала про такое существование :) Поэтому столбец geography удалила в признаках, т.к незнала как работать с текстовыми значениями.
    </div>


<div class="alert alert-danger">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Смотри, у тебя для логистической регресси не подходит LE, потому что он задаст порядок на значениях, которого изначально могло и не быть (или он был другой). Для логистической регрессии как раз подходил OHE. Вопрос был задан специально:)
</div>

In [4]:
features = churn_data.drop(['surname', 'rownumber','geography', 'customerid', 'exited'], axis=1) #признаки обучающего набора данных
target = churn_data['exited'] #целевой признак
 
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)

#обучающий набор данных
print(features_train.shape)
print(target_train.shape)

#валидационный набор данных
print(features_valid.shape)
print(target_valid.shape)

#тестовый набор данных
print(features_test.shape)
print(target_test.shape)

(6000, 12)
(6000,)
(2000, 12)
(2000,)
(2000, 12)
(2000,)


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Следует исследовать дисбаланс классов. Какой он?
</div>

<div class="alert alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:<a class="tocSkip"></a></b>
Дисбаланс у меня проверяется ниже в части 2. Исследование задачи
    </div>

### Вывод



1.  features хранит в себе признаки обучающего набора данных

Признаки:

- rownumber — индекс строки в данных
- customerId — уникальный идентификатор клиента
- creditscore — кредитный рейтинг
- gender — пол
- age — возраст
- tenure — количество недвижимости у клиента
- balance — баланс на счёте
- numofproducts — количество продуктов банка, используемых клиентом
- hascrcard — наличие кредитной карты
- isactivemember — активность клиента
- estimatedsalary — предполагаемая зарплата


2. target хранит целевой признак, который мы будем предсказывать. Это будет факт ухода клиента

Целевой признак:

- exited — факт ухода клиента. «да» — 1, «нет» — 0

Далее датасет разделила на:

- обучающий набор данных 60%
- валидационный набор данных -20%
- тестовый набор данных - 20%
- указываем случайное состояние -12345

# 2. Исследование задачи

In [5]:
def disbalans(features, target, predicted):
    print(confusion_matrix(target, predicted))
    print('Полнота:', recall_score(target,predicted))
    print('Точность:', precision_score(target,predicted))
    print("F1:", f1_score(target, predicted))
    probabilities_valid = model.predict_proba(features)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('AUC-ROC:', roc_auc_score(target, probabilities_one_valid))

In [6]:
#Accuracy для решающего дерева
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Accuracy:',accuracy_score(target_valid, predicted_valid))

disbalans(features_valid, target_valid, predicted_valid)

Accuracy: 0.7925
[[1395  214]
 [ 201  190]]
Полнота: 0.4859335038363171
Точность: 0.47029702970297027
F1: 0.4779874213836478
AUC-ROC: 0.6764658196620988


### Вывод

Качество решающего дерева соствляет 79%

Матрицу ошибок следующая:

- TN = 1398
- FP = 211
- FN = 198
- TP = 193

Доля положительных среди всех составляет 49%, доля отрицательных ответов равна 47%

F1-мера решающего дерева равна 0.48 и AUC-ROC: 0.68

In [7]:
#для логистической регрессии
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Accuracy:',accuracy_score(target_valid, predicted_valid))

disbalans(features_valid, target_valid, predicted_valid)

Accuracy: 0.692
[[1107  502]
 [ 114  277]]
Полнота: 0.7084398976982097
Точность: 0.355584082156611
F1: 0.4735042735042735
AUC-ROC: 0.7682505217613838


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
У тебя дублируется одна и та же функциональность для разных моделей, стоит описать под это функцию, которая позволит сократить дублируемость кода
</div>

### Вывод

Качество модели соствляет 69%

Матрицу ошибок следующая:

- TN = 1116
- FP = 493
- FN = 117
- TP = 274

Доля положительных среди всех составляет 70%, доля отрицательных ответов равна 35%

F1-мера для логистической регрессии равна 0.47 и AUC-ROC: 0.77

# 3. Борьба с дисбалансом

In [8]:
#Увеличение выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

model = LogisticRegression(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.35239852398523985


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
#Уменьшение выборки
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

model = LogisticRegression(random_state=12345)  
model.fit(features_downsampled, target_downsampled)  
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.3416370106761566


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
features_downsampled, target_downsampled = downsample(features_test, target_test, 0.1)
model = LogisticRegression(random_state=12345)  
model.fit(features_downsampled, target_downsampled)  
predicted_test = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_test))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


F1: 0.36324786324786323


<div class="alert alert-success">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Молодец, что попробовала разные методы борьбы с дисбалансом классов
</div>

### Вывод

- При увеличении выборки F1-мера составляет 0.35
- При уменьшении выборки F1-мера составляет 0.34

# 4. Тестирование модели

In [11]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_test))

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print('AUC-ROC:',roc_auc_score(target_test, probabilities_one_test))

F1: 0.49637389202256244
AUC-ROC: 0.7580690546413348


In [12]:
best_model_forest = None
best_result_forest = 0
best_est = 0

for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est, bootstrap = True, class_weight = 'balanced', max_depth= 7)
    model.fit(features_train, target_train) 
    
    predictions_valid = model.predict(features_valid)
    result = f1_score(target_valid, predictions_valid) 
    if result > best_result_forest:
        best_model_forest = model # сохранить наилучшую модель
        best_result_forest = result #  сохранить наилучшее значение метрики accuracy на валидационных данных
        best_est = est
        predictions_test = model.predict(features_test)
        result2 = f1_score(target_test, predictions_test)  #проверка на тестовой выборке
        probabilities_test = model.predict_proba(features_test)
        probabilities_one_test = probabilities_test[:, 1]
            
print("F1 в валидационной выборки", best_result_forest, " F1 в тестовой выборке", result2, "Количество деревьев:", best_est)

print('AUC-ROC:',roc_auc_score(target_test, probabilities_one_test))

F1 в валидационной выборки 0.573576799140709  F1 в тестовой выборке 0.6263846928499497 Количество деревьев: 10
AUC-ROC: 0.8660370925646633



<div class="alert alert-danger">
<h1>Комментарий ревьюера v3 <a class="tocSkip"></a></h1>
Но мы ведь должны подбирать гиперпараметры на валидационной выборке! На тесте подбирать не имеет смысла! (Если нужны пояснения напиши, на след итерации поясню)

+ нужно f1 как минимум 0.59, а у тебя 0.589 (не хочу душить, но заказчик жуткий формалист) :)
    
Попробуй случайный лес попробовать, может быть лучше будет
</div>

<div class="alert alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА v3:<a class="tocSkip"></a></b></div>


<div class="alert alert-success">
<h1>Комментарий ревьюера v4 <a class="tocSkip"></a></h1>
успех!</div>

### Вывод

Тестовый набор данных проверила на двух моделях.

Гармоничное среднее между точностью и полнотой лучший результат у модель случайного леса.

Максимальная глубина дерева -7 и максимальное кол-во деревьев равна 10.

F1 мера равна 0.62,  AUC-ROC: 0.86

Поэтому модель случайного леса позволит нам лучше спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.


<div class="alert alert-danger">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Хммм, а не подозрительно ли то, что у тебя модель идеально отработала на тестовх данных, нет ли data leakage? Верно ли ты тут модель обучаешь?)
</div>

<div class="alert alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА v2:<a class="tocSkip"></a></b>
невижу ничего подозрительного
    </div>


<div class="alert alert-danger">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Подумай еще немного:)  Ты ведь должна обучать на обучающей выборке
</div>

<div class="alert alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА v2:<a class="tocSkip"></a></b>
Сделала обучение с обучающей выборкой, потом уже с тестовой. Поняла где я сглупила, спасибо!

В модель с деревом решений подключила гиперпараметры с перебором наилучшего условия для того чтобы получить F1-0.59
    </div>

<div class="alert alert-danger">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Но заказчик хочет F1меру как минимум 0.59!

Попробуй применить другие модели, не всегда логистическая регрессия будет лучшим вариантом:) Также попробуй поподбирать гиперпараметры для моделей. Должно помочь. Попробуй также масштабирование признаков
    
И хочется немного побльше вывода:) Заказчик ведь не поймет такого вывода:)
</div>

<div class="alert alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:<a class="tocSkip"></a></b>
ДОбавила вторую модель для сравнния. 
    </div>


<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Посмотрел твои правки, уже лучше, думаю следующая итерация будет финальной.

Осталось совсем чуть-чуть! Жду твоих исправлений!
</div>

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>
Большое спасибо за проделанную работу. Видно, что приложено немало усилий.

- Соблюдена структура проекта 

- Код написан хорошо
    
- Я оставил несколько советов, надеюсь, что они будут учтены в будущих проектах.

- Есть критические замечания, которые нужно устранить:
    
    - RowNumber, CustomerId
    
    - Geography

    - Бейзлайн задачи
    
    - Вывод
</div>